In [ ]:
pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ray
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

ray.init()

# Define the CNN model
def build_model():
    model = keras.Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define the training function
@ray.remote
def train_cnn(X, y, model_weights):
    model = build_model()
    model.set_weights(model_weights)
    model.fit(X, y, batch_size=128, epochs=5)
    new_weights = model.get_weights()
    return new_weights

# Generate some sample data
(X_train, y_train), _ = keras.datasets.mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255
y_train = keras.utils.to_categorical(y_train, 10)

import numpy as np
# Split the data into 2 parts
X_split = np.array_split(X_train, 2)
y_split = np.array_split(y_train, 2)

# Train the CNN model on 2 different nodes
num_nodes = 2
model = build_model()
model_weights = model.get_weights()
node_weights = ray.get([train_cnn.remote(X_split[j], y_split[j], model_weights) for j in range(num_nodes)])
for i in range(num_nodes):
    print(f"Node {i}: {node_weights[i]}")

# Combine the weights from the different nodes
new_weights = []
for i in range(len(model_weights)):
    layer_weights = []
    for j in range(num_nodes):
        layer_weights.append(node_weights[j][i])
    new_weights.append(np.mean(layer_weights, axis=0))
model.set_weights(new_weights)

# Evaluate the model on the test data
(_, _), (X_test, y_test) = keras.datasets.mnist.load_data()
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255
y_test = keras.utils.to_categorical(y_test, 10)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test accuracy: {test_acc}")


2023-03-28 19:51:27,764	INFO worker.py:1553 -- Started a local Ray instance.
(pid=3080) 2023-03-28 19:51:32.521222: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
(pid=3080) 2023-03-28 19:51:32.521390: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
(pid=3080) 2023-03-28 19:51:32.521415: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
(

(train_cnn pid=3079) Epoch 1/5
(train_cnn pid=3080) Epoch 1/5
235/235 [==============================] - 65s 265ms/step - loss: 0.4520 - accuracy: 0.8782
(train_cnn pid=3079) Epoch 2/5
235/235 [==============================] - 64s 261ms/step - loss: 0.4545 - accuracy: 0.8808
(train_cnn pid=3080) Epoch 2/5
235/235 [==============================] - 62s 264ms/step - loss: 0.1068 - accuracy: 0.9692
(train_cnn pid=3079) Epoch 3/5
235/235 [==============================] - 63s 268ms/step - loss: 0.1091 - accuracy: 0.9684
(train_cnn pid=3080) Epoch 3/5
235/235 [==============================] - 71s 300ms/step - loss: 0.0720 - accuracy: 0.9780
(train_cnn pid=3079) Epoch 4/5
235/235 [==============================] - 70s 298ms/step - loss: 0.0731 - accuracy: 0.9787
(train_cnn pid=3080) Epoch 4/5
235/235 [==============================] - 69s 295ms/step - loss: 0.0564 - accuracy: 0.9832
(train_cnn pid=3079) Epoch 5/5
235/235 [==============================] - 69s 295ms/step - loss: 0.0579 - ac